# Circuit learning module: Pennylane with SPSA

This code is inspired by [Quanthoven](https://github.com/CQCL/Quanthoven/blob/main/experiment.ipynb) and [Pennylane demo](https://pennylane.ai/qml/demos/spsa.html).

In [1]:
import json
import os
import sys
import glob
import warnings
import collections
from pathlib import Path
import numpy as np
import pickle
import pennylane as qml
from sympy import default_sort_key
import torch
from discopy.quantum.pennylane import to_pennylane, PennyLaneCircuit
from inspect import signature
from noisyopt import minimizeSPSA
from utils import transform_into_pennylane_circuits, read_diagrams, get_symbols, create_labeled_classes, acc_from_dict, loss_from_dict

this_folder = os.path.abspath(os.getcwd())
nshot = 10000

In [2]:
def genbin(n, bs=''):
    if len(bs) == n:
        return bs
    else:
        return np.array([genbin(n, bs + '0'), genbin(n, bs + '1')]).flatten()

## Read circuit data

We read the circuits from the pickled files.

In [3]:
# Select workload
#workload = "execution_time"
workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//"\
            + workload + "//" + workload_size + "//circuits//"\
            + str(classification) + "//" + str(layers) + "_layer//"\
           + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count)\
            + "_n_wire_count//"

In [4]:
training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [5]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [6]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Constructing Pennylane circuits

In [7]:
qml_training_circuits, train_symbols = transform_into_pennylane_circuits(training_circuits)
qml_test_circuits, test_symbols = transform_into_pennylane_circuits(test_circuits)
qml_validation_circuits, val_symbols = transform_into_pennylane_circuits(validation_circuits)

In [8]:
fix = []
for i in qml_training_circuits:
    if i not in training_data_labels:
        fix.append(i)
        
for i in fix:
    qml_training_circuits.pop(i, None)

fix = []
for i in qml_validation_circuits:
    if i not in validation_data_labels:    
        fix.append(i)

for i in fix:
    qml_validation_circuits.pop(i, None)

fix = []
for i in qml_test_circuits:
    if i not in test_data_labels:
        fix.append(i)
        
for i in fix:
    qml_test_circuits.pop(i, None)

In [9]:
#training_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_training_circuits.values()])
#validation_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_validation_circuits.values()])
#test_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_test_circuits.values()])

In [10]:
print("Number of training circuits: ", len(qml_training_circuits))
print("Number of validation circuits: ", len(qml_validation_circuits))
print("Number of test circuits: ", len(qml_test_circuits))

Number of training circuits:  440
Number of validation circuits:  108
Number of test circuits:  109


## Post-selection

In [11]:
def post_select_circuit_samples(circuit_samples, n_qubits, post_selection):
    selected_samples = []
    post_select_array = np.array([0]*(n_qubits - post_selection))
    for circuit_sample in circuit_samples:
        if np.array_equal(circuit_sample[post_selection - 1 :-1], post_select_array):
            res = circuit_sample[0:post_selection]
            selected_samples.append(res)
            #if circuit_sample[0] == 1:
            #    selected_samples.append(1)
            #else:
            #    selected_samples.append(0)
    return selected_samples

## Cost and prediction functions

Because Pennylane does not implement post-selection, we need to implement it as we would implement it on a real quantum hardware i.e. using sampling.

In [12]:
def make_pred_fn(circuits):
    
    def predict2(params):
        predictions = {}
        for c in circuits:
            disco_circuit = circuits[c]
            qml_circuit = to_pennylane(disco_circuit)
            result = qml_circuit.post_selected_circuit(params)
            predictions[c] = result
        return predictions
    
    def predict(params):
        predictions = {}
        i = 0
        for c in circuits:
            circuit_fun = qnodes[q]["circuit_fun"]
            n_qubits = qnodes[q]["n_qubits"]
            dev = qml.device("default.qubit", wires=n_qubits)
            circuit = qml.QNode(circuit_fun, dev)
            res = []
            while sum(res) < 1:
                res = []
                measurement = circuit(params)
                post_selected_samples = post_select_circuit_samples(measurement, n_qubits, 2)
                post_selected_samples = [tuple(map(int, t)) for t in post_selected_samples]
                counts = collections.Counter(post_selected_samples)
                for s in genbin(classification):
                    t = tuple(map(int, s))
                    if t in counts:
                        res.append(counts[t]/len(post_selected_samples))
                    else:
                        res.append(1e-9)
            predictions[q] = np.array(res)
            i+=1
            if i == 4:
                break
        return predictions
        
    return predict

In [13]:
def make_cost_fn(pred_fn, labels):
    
    def cost_spsa(params, **kwargs):
        
        predictions = pred_fn(params)
        #print(predictions)
        
        cost = loss_from_dict(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc_from_dict(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)
        
        return cost
    
    costs, accuracies = [], []
    return cost_spsa, costs, accuracies

## SPSA minimization

In [24]:
EPOCHS = 100
SEED = 0
result_file = workload + "_" + workload_size + "_pennylane_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

#np.random.seed(SEED)
rng = np.random.default_rng(SEED)
init_params_spsa = np.array(rng.random(len(train_symbols)))
#init_params_spsa = torch.tensor(init_params_spsa)

train_pred_func = make_pred_fn(qml_training_circuits)
dev_pred_func = make_pred_fn(qml_validation_circuits)
test_pred_func = make_pred_fn(qml_test_circuits)

train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_func, training_data_labels)
dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_func, validation_data_labels)
test_cost_fn, test_costs, test_accs = make_cost_fn(test_pred_func, test_data_labels)

In [27]:
#print(init_params_spsa)

In [36]:
disco_circuit = training_circuits["101"]
qml_circuit = to_pennylane(disco_circuit, probabilities = True)
symbols = disco_circuit.free_symbols
symbols = list(sorted(symbols, key=default_sort_key))
init_params_spsa = torch.Tensor([[t] for t in np.array(rng.random(len(symbols)))])
print(len(symbols) == len(init_params_spsa))
result = qml_circuit.eval(symbols=symbols, weights=init_params_spsa)
print(result)

True
tensor([[0.0003, 0.0002],
        [0.0007, 0.0005]], dtype=torch.float64)


In [ ]:
def callback_fn(xk):
    valid_loss = dev_cost_fn(xk)
    train_loss = np.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
    train_acc = np.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
    valid_acc = np.around(float(dev_accs[-1]), 4)
    iters = int(len(train_accs)/2)
    #if iters % 200 == 0:
    info = f"Epoch: {iters}   "\
            + f"train/loss: {train_loss}   "\
            + f"valid/loss: {np.around(float(valid_loss), 4)}   "\
            + f"train/acc: {train_acc}   "\
            + f"valid/acc: {valid_acc}"

    with open("results//" + result_file + ".txt", "a") as f:
        f.write(info + "\n")

    print(info, file=sys.stderr)
    
    return valid_loss